# Part 4: Grouping and Sorting

In [9]:
import pandas as pd
reviews = pd.read_csv("./resources/wine-reviews/winemag-data-130k-v2.csv", index_col=0)
pd.set_option("display.max_rows", 5)

## Groupwise analysis

- We can group data using the <mark>groupby()</mark> function

In [4]:
reviews.groupby('points').points.count()

points
80     397
81     692
      ... 
99      33
100     19
Name: points, Length: 21, dtype: int64

- <mark>groupby()</mark> created a group of reviews with the same <mark>points</mark>
- We then counted the number of elements for each group, similar to <mark>SQL</mark>
- <mark>value_counts()</mark> is a shortcut for this

In [5]:
reviews.groupby('points').price.min()

points
80      5.0
81      5.0
       ... 
99     44.0
100    80.0
Name: price, Length: 21, dtype: float64

- We can also use <mark>apply()</mark> for further manipulation

In [6]:
reviews.groupby('winery').apply(lambda df: df.iloc[0])

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
winery,,,,,,,,,,,,,
1+1=3,Spain,"A dusty, yeasty nose is simplistic but friendl...",Rosé,86,20.0,Catalonia,Cava,NaN,Michael Schachner,@wineschach,1+1=3 NV Rosé Sparkling (Cava),Sparkling Blend,1+1=3
10 Knots,US,"A strongly flavored, sugary, unsubtle wine, le...",NaN,84,22.0,California,Paso Robles,Central Coast,NaN,NaN,10 Knots 2010 Viognier (Paso Robles),Viognier,10 Knots
...,...,...,...,...,...,...,...,...,...,...,...,...,...
àMaurice,US,"Bright aromas of freshly roasted coffee beans,...",Fred Estate,89,45.0,Washington,Walla Walla Valley (WA),Columbia Valley,Sean P. Sullivan,@wawinereport,àMaurice 2013 Fred Estate Syrah (Walla Walla V...,Syrah,àMaurice
Štoka,Slovenia,"Vanilla and cookie crumb notes add a sweet, to...",Izbrani,88,20.0,Kras,NaN,NaN,Anna Lee C. Iijima,NaN,Štoka 2009 Izbrani Teran (Kras),Teran,Štoka


- We can also group by more than one column

In [7]:
reviews.groupby(['country', 'province']).apply(lambda df: df.loc[df.points.idxmax()])

country  ...                winery
country   province                     ...                      
Argentina Mendoza Province  Argentina  ...  Bodega Catena Zapata
          Other             Argentina  ...                Colomé
...                               ...  ...                   ...
Uruguay   San Jose            Uruguay  ...        Castillo Viejo
          Uruguay             Uruguay  ...               Narbona

[425 rows x 13 columns]

- <mark>agg()</mark> can be used to run different functions simultaneously

In [8]:
reviews.groupby('country').price.agg([len, min, max])

,len,min,max
country,,,
Argentina,3800,4.0,230.0
Armenia,2,14.0,15.0
...,...,...,...
Ukraine,14,6.0,13.0
Uruguay,109,10.0,130.0


## Multi-indexes

- So far, every <mark>DataFrame</mark> and <mark>Series</mark> had a single-label index
- <mark>groupby()</mark> sometimes results in a multi-index
- A multi-index has multiple levels

In [10]:
countries_reviewed = reviews.groupby(['country', 'province']).description.agg([len])
countries_reviewed

len
country   province              
Argentina Mendoza Province  3264
          Other              536
...                          ...
Uruguay   San Jose             3
          Uruguay             24

[425 rows x 1 columns]

In [11]:
mi = countries_reviewed.index
type(mi)

pandas.core.indexes.multi.MultiIndex

- Multi-indices require two levels of labels to retrieve a value
- Multi-indices are an advanced concept in pandas
- We can convert multi-indices back to regular indices by <mark>reset_index()</mark>

In [12]:
countries_reviewed.reset_index()

,country,province,len
0,Argentina,Mendoza Province,3264
1,Argentina,Other,536
...,...,...,...
423,Uruguay,San Jose,3
424,Uruguay,Uruguay,24


## Sorting
- <mark>groupby()</mark> returns data in index order
- We can sort the data with <mark>sort_values()</mark> the way we want

In [13]:
countries_reviewed = countries_reviewed.reset_index()
countries_reviewed.sort_values(by='len')

,country,province,len
179,Greece,Muscat of Kefallonian,1
192,Greece,Sterea Ellada,1
...,...,...,...
415,US,Washington,8639
392,US,California,36247


- <mark>sort_values()</mark> sorts in ascending order per default

In [14]:
countries_reviewed.sort_values(by='len', ascending=False)

,country,province,len
392,US,California,36247
415,US,Washington,8639
...,...,...,...
63,Chile,Coelemu,1
149,Greece,Beotia,1


- To sort by index values, we can use <mark>sort_index()</mark>

In [15]:
countries_reviewed.sort_index()

,country,province,len
0,Argentina,Mendoza Province,3264
1,Argentina,Other,536
...,...,...,...
423,Uruguay,San Jose,3
424,Uruguay,Uruguay,24


- We can also sort by more than one column

In [16]:
countries_reviewed.sort_values(by=['country', 'len'])

,country,province,len
1,Argentina,Other,536
0,Argentina,Mendoza Province,3264
...,...,...,...
424,Uruguay,Uruguay,24
419,Uruguay,Canelones,43


## Exercises

In [17]:
reviews = pd.read_csv("./resources/wine-reviews/winemag-data-130k-v2.csv", index_col=0)

### 1.

In [21]:
reviews_written = reviews.groupby('taster_twitter_handle').size()
reviews_written

taster_twitter_handle
@AnneInVino        3685
@JoeCz             5147
                   ... 
@winewchristina       6
@worldwineguys     1005
Length: 15, dtype: int64

### 2.

In [28]:
best_rating_per_price = reviews.groupby('price').points.max().sort_index()
best_rating_per_price

price
4.0       86
5.0       87
          ..
2500.0    96
3300.0    88
Name: points, Length: 390, dtype: int64

### 3.

In [32]:
price_extremes = reviews.groupby('variety').price.agg([min, max])
price_extremes

,min,max
variety,,
Abouriou,15.0,75.0
Agiorgitiko,10.0,66.0
...,...,...
Çalkarası,19.0,19.0
Žilavka,15.0,15.0


### 4.

In [34]:
sorted_varieties = price_extremes.sort_values(['min', 'max'], ascending=False)
sorted_varieties

,min,max
variety,,
Ramisco,495.0,495.0
Terrantez,236.0,236.0
...,...,...
Vital,NaN,NaN
Zelen,NaN,NaN


### 5.

In [38]:
reviewer_mean_ratings = reviews.groupby('taster_name').points.mean()
reviewer_mean_ratings

taster_name
Alexander Peartree    85.855422
Anna Lee C. Iijima    88.415629
                        ...    
Susan Kostrzewa       86.609217
Virginie Boone        89.213379
Name: points, Length: 19, dtype: float64

In [39]:
reviewer_mean_ratings.describe()

count    19.000000
mean     88.233026
           ...    
75%      88.975256
max      90.562551
Name: points, Length: 8, dtype: float64

### 6.

In [45]:
country_variety_counts = reviews.groupby(['country', 'variety']).size().sort_values(ascending=False)
country_variety_counts

country  variety           
US       Pinot Noir            9885
         Cabernet Sauvignon    7315
                               ... 
Mexico   Rosado                   1
Uruguay  White Blend              1
Length: 1612, dtype: int64